# Data Masking Sample 
### Using StackOverflow2010 Database from BrentOzar.com

My task here is to identify the sensitivity of the database, so that I can capture that in SQL Data Catalog (my system of record for data classification), then use that metadata to define my masking operations before secondary use of that database.

In [19]:
-- Get a list of tables and views in the current database
SELECT table_catalog [database], table_schema [schema], table_name [name], table_type [type]
FROM [StackOverflow2010].INFORMATION_SCHEMA.TABLES;

: Query failed: The connection does not support MultipleActiveResultSets.

Total execution time: 00:00:00

First, let's look at the Users table before masking. Some recognisable names here. Location is pretty specific in some cases, and AboutMe is sometimes very identifiable. WebsiteUrl definitely so. EmailHash is null in all cases. What if it started to be used though? This is something I would flag up for refactoring; if it's not used, let's not leave it in the schema. Sometimes there's some common-sense risk reduction that we can use this opportunity to implement.

In [20]:
SELECT Top(10) * FROM [StackOverflow2010].dbo.Users;

: Query failed: The connection does not support MultipleActiveResultSets.

Total execution time: 00:00:00

Next, the Badges table.

In [21]:
SELECT Top(5) * FROM [StackOverflow2010].dbo.Badges;

: Query failed: The connection does not support MultipleActiveResultSets.

Total execution time: 00:00:00

Slightly odd normalisation decision perhaps, but not sensitive.
OK, leave that one, Votes next.

In [22]:
SELECT Top(5) * FROM [StackOverflow for Masking].dbo.Votes;

: Query failed: The connection does not support MultipleActiveResultSets.

Total execution time: 00:00:00

Can't see anything there. If we've masked the users well, what they've voted for won't be very interesting. Or _will_ it? If I know the context, I might be able to do reidentification. Jon Skeet is famously prolific for example. One of those slightly tricky calls that requires business context and a risk assessment. Who is going to see the masked database? What would happen id they reidentified using their knowledge?

In [23]:
SELECT Top(5) * FROM [StackOverflow2010].dbo.Comments;

: Query failed: The connection does not support MultipleActiveResultSets.

Total execution time: 00:00:00.001

Going to need to handle that text I think. Ok, Posts next.

In [24]:
SELECT Top(5) * FROM [StackOverflow2010].dbo.Posts;

: Query failed: The connection does not support MultipleActiveResultSets.

Total execution time: 00:00:00.001

LastEditorDisplayName, going on this list.
How about Body? This is a bit trickier; the body is in most cases a public post on a website about a technical matter. How could that be confidential?
But here's the rub; GDPR isn't necessarily about confidentiality. It's about taking a risk-based approach to the handling of personal data. The first thing we have to identify is whether it is personal. 
```
Personal data is information that relates to an identified or identifiable individual.

What identifies an individual could be as simple as a name or a number or could include other identifiers such as an IP address or a cookie identifier, or other factors.
```
https://ico.org.uk/for-organisations/guide-to-data-protection/guide-to-the-general-data-protection-regulation-gdpr/key-definitions/what-is-personal-data/

In [15]:
SELECT Top(5) * FROM [StackOverflow2010].dbo.Posts where Body like '%@%';

: Query failed: The connection does not support MultipleActiveResultSets.

Total execution time: 00:00:00

There are references to other people, that looks like someone's personal blog url there as well. This is identifiable data. It might be low in risk and confidentiality, but that's an assessment for later.